## Installing the Library

In [ ]:
pip install openpyxl

In [ ]:
pip install pandas

In [ ]:
pip install datetime

## Importing the Libraries 

In [ ]:
import pandas as pd
import datetime
import openpyxl as xl
from openpyxl.styles import Font, NamedStyle, Alignment
from openpyxl.utils.cell import get_column_letter

## Load the source workbook and destination workbook

In [ ]:
wb_source_filename = 'Input File - Dummy.xlsx'
wb_source = xl.load_workbook(wb_source_filename)

wb_dest_filename = 'GS India Equity Portfolio Positioning and Holdings_Feb-23.xlsx'
wb_dest = xl.load_workbook(wb_dest_filename)

wb_stocks_filename = 'Stock Name Database.xlsx'
wb_stocks = xl.load_workbook(wb_stocks_filename)


## Source Start & End Row

In [ ]:
# Please provide the starting row and ending row values for the source. eg:(10 , 20)

sector_source_start_row, sector_source_end_row = 8, 20

market_cap_source_start_row, market_cap_source_end_row = 8, 11

top_10_holding_source_start_row, top_10_holding_source_end_row = 8, 123

top_10_overweight_source_start_row, top_10_overweight_source_end_row = 8, 122

currency_source_start_row, currency_source_end_row = 8, 34

holding_source_start_row, holding_source_end_row = 8, 124

## Positioning 

In [ ]:
check_list = []

## Sector

In [ ]:
# Select the source and destination worksheets
ws_source = wb_source['Sector']
ws_dest = wb_dest['Positioning']

source_start_row = sector_source_start_row
source_end_row = sector_source_end_row
source_row = (source_end_row - source_start_row) + 1

dest_start_row = 7
dest_end_row = 19 
dest_row = (dest_end_row - dest_start_row) + 1 

check_row = source_row - dest_row 
check_list.append(check_row)

if check_row > 0:
    ws_dest.insert_rows(dest_start_row, amount=check_row)
elif check_row < 0:
    ws_dest.delete_rows(dest_start_row, amount= abs(check_row))
    
for row in [6, 23, 32, 45, 66]:
    ws_dest.row_dimensions[row].height = 15
    
row_number = dest_start_row - 1
ws_dest.row_dimensions[row_number].height = 28.20

# Define the rows you want to iterate over in the source worksheet
rows_source = ws_source.iter_rows(min_row=source_start_row, max_row=source_end_row, max_col=4)

# Define the rows you want to iterate over in the destination worksheet
rows_dest = ws_dest.iter_rows(min_row=dest_start_row, max_row=dest_end_row + check_row, max_col=4)

# Define the font style
font_style = Font(name='Arial', size=10, bold=True, color='003366')

# Copy data from the source to the destination worksheet and apply font style
for row_source, row_dest in zip(rows_source, rows_dest):
    for cell_source, cell_dest in zip(row_source, row_dest):
        cell_dest.value = cell_source.value
        cell_dest.font = font_style

# Save the changes to the destination workbook
wb_dest.save(wb_dest_filename)


## Market Cap

In [ ]:
# Select the source and destination worksheets
ws_source = wb_source['Mcap']
ws_dest = wb_dest['Positioning']

source_start_row = market_cap_source_start_row
source_end_row = market_cap_source_end_row
source_row = (source_end_row - source_start_row) + 1

dest_start_row = 24 + sum(check_list)
dest_end_row = 28 + sum(check_list)
dest_row = (dest_end_row - dest_start_row) + 1

check_row = source_row - dest_row # -1
check_list.append(check_row)

if check_row > 0:
    ws_dest.insert_rows(dest_start_row, amount=check_row)
elif check_row < 0:
    ws_dest.delete_rows(dest_start_row, amount= abs(check_row))
    
row_number = dest_start_row - 1
ws_dest.row_dimensions[row_number].height = 28.20

# Define the rows you want to iterate over in the source worksheet
rows_source = ws_source.iter_rows(min_row=source_start_row, max_row=source_end_row, max_col=4)

# Define the rows you want to iterate over in the destination worksheet
rows_dest = ws_dest.iter_rows(min_row=dest_start_row, max_row=dest_end_row + check_row, max_col=4)

# Define the font style
font_style = Font(name='Arial', size=10, bold=True, color='003366')

# Copy data from the source to the destination worksheet and apply font style
for row_source, row_dest in zip(rows_source, rows_dest):
    for cell_source, cell_dest in zip(row_source, row_dest):
        cell_dest.value = cell_source.value
        cell_dest.font = font_style

# Save the changes to the destination workbook
wb_dest.save(wb_dest_filename)

## Top 10 Holdings

In [ ]:
# Specify the input file path
input_excel_path = "Input File - Dummy.xlsx"
sheet_name = "Top 10 HS" 
ws_dest = wb_dest['Positioning']

# Specify the rows you want to read (e.g., rows 1 to 10)
source_start_row = top_10_holding_source_start_row 
source_end_row = top_10_holding_source_end_row    
columns_to_select = [0, 2, 3, 4, 5] # 0 based index

dest_start_row = 33 + sum(check_list)
dest_end_row = 42 + sum(check_list)

row_number = dest_start_row - 1
ws_dest.row_dimensions[row_number].height = 28.20

# Generate a timestamp to create a unique output filename
timestamp = datetime.datetime.now().strftime("%M%S")
output_excel_path = f"Dummy\\Top_10_Holding_Dummy{timestamp}.xlsx"


# Read the Excel file
df = pd.read_excel(input_excel_path,sheet_name=sheet_name,usecols=columns_to_select, 
                   skiprows=range(1, source_start_row), nrows=source_end_row - source_start_row + 1)


# Convert the values in the column to float
df.iloc[:, 2] = df.iloc[:, 2].astype(float)
df.iloc[:, 3] = df.iloc[:, 3].astype(float)
df.iloc[:, 4] = df.iloc[:, 4].astype(float)
df = df.sort_values(by=df.columns[2], ascending=False)

# Remove rows where the 1st column value is "[Cash]"
df = df[df.iloc[:, 0] != "[Cash]"]

# Select only the first 10 rows
df = df.head(10)

# Define the font style
font_style = Font(name='Arial', size=10, bold=True, color='003366')

# Write the data to a new Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)

wb_source_top_10_holdings = xl.load_workbook(output_excel_path)
ws_source = wb_source_top_10_holdings['Sheet1']
rows_source = ws_source.iter_rows(min_row=2, max_row=11)

# Define the rows you want to iterate over in the destination worksheet
rows_dest = ws_dest.iter_rows(min_row=dest_start_row, max_row=dest_end_row)

# Copy data from the source to the destination worksheet
for row_source, row_dest in zip(rows_source, rows_dest):
    for cell_source, cell_dest in zip(row_source, row_dest):
        cell_dest.value = cell_source.value
        cell_dest.font = font_style

# Save the changes to the destination workbook
wb_dest.save(wb_dest_filename)

## Top 10 OverWeight

In [ ]:
# Specify the input file path
input_excel_path = "Input File - Dummy.xlsx"
sheet_name = "Top 10 OW" 
ws_dest = wb_dest['Positioning']

# Specify the rows you want to read (e.g., rows 1 to 10)
source_start_row = top_10_overweight_source_start_row
source_end_row = top_10_overweight_source_end_row    
columns_to_select = [0, 2, 3, 4, 5]

dest_start_row = 46 + sum(check_list)
dest_end_row = 55 + sum(check_list)

row_number = dest_start_row - 1
ws_dest.row_dimensions[row_number].height = 28.20

# Generate a timestamp to create a unique output filename
timestamp = datetime.datetime.now().strftime("%M%S")
output_excel_path = f"Dummy\\Top_10_OverWeight_Dummy{timestamp}.xlsx"

# Read the Excel file
df = pd.read_excel(input_excel_path,sheet_name=sheet_name,usecols=columns_to_select, 
                   skiprows=range(1, source_start_row), nrows=source_end_row - source_start_row + 1)

# Convert the values in the column to float
df.iloc[:, 2] = df.iloc[:, 2].astype(float)
df.iloc[:, 3] = df.iloc[:, 3].astype(float)
df.iloc[:, 4] = df.iloc[:, 4].astype(float)
df = df.sort_values(by=df.columns[4], ascending=False)

# Remove rows where the 4th column value is "[Cash]"
df = df[df.iloc[:, 0] != "[Cash]"]

# Select only the first 10 rows
df = df.head(10)

# Define the font style
font_style = Font(name='Arial', size=10, bold=True, color='003366')

# Write the data to a new Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)

wb_source_top_10_overweight = xl.load_workbook(output_excel_path)
ws_source = wb_source_top_10_overweight['Sheet1']
rows_source = ws_source.iter_rows(min_row=2, max_row=11)

# Define the rows you want to iterate over in the destination worksheet
rows_dest = ws_dest.iter_rows(min_row=dest_start_row, max_row=dest_end_row)

# Copy data from the source to the destination worksheet
for row_source, row_dest in zip(rows_source, rows_dest):
    for cell_source, cell_dest in zip(row_source, row_dest):
        cell_dest.value = cell_source.value
        cell_dest.font = font_style

# Save the changes to the destination workbook
wb_dest.save(wb_dest_filename)

## Currency 

In [ ]:
# Select the source and destination worksheets
ws_source = wb_source['Currency']
ws_dest = wb_dest['Positioning']

source_start_row = currency_source_start_row
source_end_row = currency_source_end_row
source_row = (source_end_row - source_start_row) + 1 

dest_start_row = 59 + sum(check_list)
dest_end_row = 60 + sum(check_list)
dest_row = (dest_end_row - dest_start_row) + 1

check_row = source_row - dest_row  
check_list.append(check_row)

if check_row > 0:
    ws_dest.insert_rows(dest_start_row, amount=check_row)
elif check_row < 0:
    ws_dest.delete_rows(dest_start_row, amount= abs(check_row))
    
row_number = dest_start_row - 1
ws_dest.row_dimensions[row_number].height = 28.20
    
# Define the rows you want to iterate over in the source worksheet
rows_source = ws_source.iter_rows(min_row=source_start_row, max_row=source_end_row, max_col=2)

# Define the rows you want to iterate over in the destination worksheet
rows_dest = ws_dest.iter_rows(min_row=dest_start_row, max_row=dest_end_row + check_row, max_col=2)

# Define the font style
font_style = Font(name='Arial', size=10, bold=True, color='003366')

# Copy data from the source to the destination worksheet and apply font style
for row_source, row_dest in zip(rows_source, rows_dest):
    for cell_source, cell_dest in zip(row_source, row_dest):
        cell_dest.value = cell_source.value
        cell_dest.font = font_style

style = NamedStyle(name='decimal_style')
style.format = '0.0'
center_alignment = Alignment(horizontal="center", vertical="center")

for i in range(dest_start_row, dest_end_row + check_row + 1):
    ws_dest['B'+str(i)].style = style
    ws_dest['B'+str(i)].font = font_style
    ws_dest['B'+str(i)].alignment = center_alignment
    
# Save the changes to the destination workbook
wb_dest.save(wb_dest_filename)

## Holdings

In [ ]:
# Specify the input file path
input_excel_path = wb_source_filename
sheet_name = "Emerging" 
ws_dest = wb_dest['Holdings']

# Specify the rows you want to read (e.g., rows 1 to 10)
source_start_row = holding_source_start_row - 1
source_end_row = holding_source_end_row - 1    
source_row = (source_end_row - source_start_row) + 1 

dest_start_row = 8
dest_end_row = 107
dest_row = (dest_end_row - dest_start_row) + 1  

check_row = source_row - dest_row  

if check_row > 0:
    ws_dest.insert_rows(dest_start_row + 1, amount=check_row)
elif check_row < 0:
    ws_dest.delete_rows(dest_start_row + 1, amount= abs(check_row))

columns_to_select = [0, 1, 2, 3, 4, 6, 7, 8]

# Generate a timestamp to create a unique output filename
timestamp = datetime.datetime.now().strftime("%M%S")
output_excel_path = f"Dummy\\Holdings_Dummy{timestamp}.xlsx"

# Read the Excel file
df = pd.read_excel(input_excel_path, sheet_name = sheet_name,usecols = columns_to_select, 
                   skiprows=range(1, source_start_row), nrows = source_end_row - source_start_row + 1)

# Convert the values in the column to float
df.iloc[:, 5] = df.iloc[:, 5].astype(float)
df.iloc[:, 6] = df.iloc[:, 6].astype(float)
df.iloc[:, 7] = df.iloc[:, 7].astype(float)

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)

df1 = pd.read_excel(input_excel_path, sheet_name=sheet_name,usecols=[7, 8], skiprows=range(1, source_start_row), nrows=1)

ws_dest.cell(row=dest_start_row - 1, column=8, value=df1.iat[0, 0])
ws_dest.cell(row=dest_start_row - 1, column=9, value=df1.iat[0, 1])

wb_source_holdings = xl.load_workbook(output_excel_path)
ws_source = wb_source_holdings['Sheet1']
rows_source = ws_source.iter_rows(min_row=3)

# Define the rows you want to iterate over in the destination worksheet
rows_dest = ws_dest.iter_rows(min_row = dest_start_row, max_row=dest_end_row + check_row, min_col = 2, max_col = 9)

# Copy data from the source to the destination worksheet
for row_source, row_dest in zip(rows_source, rows_dest):
    for cell_source, cell_dest in zip(row_source, row_dest):
        cell_dest.value = cell_source.value
        cell_dest.font = Font(name='Arial')

style = NamedStyle(name='decimal_style')
style.format = '0.0'
center_alignment = Alignment(horizontal="center", vertical="center")
        
ws_source_1 = wb_source[sheet_name]

# Calculate the final line of excel data
row_index = source_end_row + 2   
count = 0
while True:
    cell_value =ws_source_1['A' + str(row_index)].value
    count += 1
    if cell_value == 'Total':
        # If the value is 'Total', break the loop
        break
    row_index += 1 
    
rows_source_1 = ws_source_1.iter_rows(min_row= source_end_row + 2, max_row=row_index, min_col=8, max_col=9)
rows_dest_1 = ws_dest.iter_rows(min_row = dest_end_row + check_row + 1, min_col = 8, max_col = 9)

for row_source, row_dest in zip(rows_source_1, rows_dest_1):
    for cell_source, cell_dest in zip(row_source, row_dest):
        cell_dest.value = cell_source.value
        cell_dest.font = Font(name='Arial')
        cell_dest.alignment = center_alignment
        
for i in range(dest_start_row, dest_end_row + check_row + 1):
    for j in ['G', 'H', 'I']:
        ws_dest[j+str(i)].alignment = center_alignment
ws_dest['H7'].alignment = center_alignment

for k in range(1,count + 1): 
    for col in ['H', 'I']:
        if k == 1 | k == count - 1:
            ws_dest[col + str(dest_end_row + check_row + k)].font = Font(name='Arial', size=10, bold=True, color='003366')
        elif k == count:
            ws_dest[col + str(dest_end_row + check_row + k)].font = Font(name='Arial', size=10, bold=True, color='000000')
    

# Save the changes to the destination workbook
wb_dest.save(wb_dest_filename)

## Name Convention

In [ ]:
sheet = wb_stocks["stock name database"]
dict_data = {"[Cash]": "Cash"}
holding_dict_data = {"[Cash]": "Cash"}
for row in sheet.iter_rows(values_only=True):
    if len(row) >= 2:
        key, value = row[:2]
        dict_data[key] = value

for worksheet in ['Positioning', 'Holdings']:
    sheet = wb_dest[worksheet]
    number_rows = sheet.max_row
    number_columns = sheet.max_column
    if worksheet == 'Holdings':
        replacement = holding_dict_data
    else:
        replacement = dict_data
    for i in range(number_columns):
        for k in range(number_rows):
            cell = str(sheet[get_column_letter(i+1)+str(k+1)].value)
            for key in replacement.keys():
                if str(cell) == key:
                    newCell = replacement.get(key)
                    sheet[get_column_letter(i+1)+str(k+1)] = str(newCell)
                
wb_dest.save(wb_dest_filename)